In [8]:
# --- 1. 라이브러리 설치 및 임포트 ---

# .env 파일을 사용하기 위한 라이브러리를 설치합니다.
!pip install python-dotenv

import pandas as pd
import requests
import time
from tqdm import tqdm
import os
from dotenv import load_dotenv # .env 파일을 읽기 위한 라이브러리

# tqdm 라이브러리와 pandas의 연동을 설정합니다.
tqdm.pandas()

In [9]:
# --- 2. .env 파일에서 API 키 로드 ---

# .env 파일에 정의된 환경 변수를 현재 세션으로 로드합니다.
load_dotenv()

# os.getenv() 함수를 사용하여 "KAKAO_REST_API_KEY" 라는 이름의 환경 변수 값을 가져옵니다.
KAKAO_REST_API_KEY = os.getenv("KAKAO_REST_API_KEY")

print("KAKAO_REST_API_KEY:", KAKAO_REST_API_KEY)

KAKAO_REST_API_KEY: f760c2b9b524c0fc455b78fd2a58b773


In [10]:
# --- 3. 설정 및 데이터 로드 ---

TRAIN_DATA_PATH = '../data/raw/train.csv'
OUTPUT_DIR = '../data/processed/'
OUTPUT_FILENAME = 'train_coordinate_geocoded.csv'
OUTPUT_PATH = os.path.join(OUTPUT_DIR, OUTPUT_FILENAME)

print(f"'{TRAIN_DATA_PATH}'에서 원본 데이터를 로드합니다...")
try:
    df = pd.read_csv(TRAIN_DATA_PATH)
    print("데이터 로드 완료.")
    initial_missing_count = df['좌표X'].isnull().sum()
    print(f"작업 전 '좌표X'의 결측치 개수: {initial_missing_count}")
except FileNotFoundError:
    print(f"오류: '{TRAIN_DATA_PATH}' 파일을 찾을 수 없습니다.")
    df = None

'../data/raw/train.csv'에서 원본 데이터를 로드합니다...


/tmp/ipykernel_87372/4207247412.py:10: DtypeWarning: Columns (16,17,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(TRAIN_DATA_PATH)


데이터 로드 완료.
작업 전 '좌표X'의 결측치 개수: 869670


In [15]:
# --- 4. 지오코딩 함수 정의 ---

def get_coordinates_kakao(address, api_key):
    """
    카카오 API를 활용하여 한글 주소(법정동+번지 등) → 좌표(경도, 위도) 변환
    """
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    params = {"query": address}
    try:
        response = requests.get(url, headers=headers, params=params, timeout=3)
        response.raise_for_status()
        data = response.json()
        if data['documents']:
            lon = float(data['documents'][0]['x'])
            lat = float(data['documents'][0]['y'])
            return lon, lat
        else:
            return None, None
    except Exception as e:
        print(f"[API오류] 주소 변환 중 에러: {e} (address: {address})")
        return None, None

In [ ]:
# --- 4-1. 테스트용 샘플 1건 선택 ---

# 데이터 불러오기
train_df = pd.read_csv('../data/raw/train.csv')

test_row = train_df[train_df['좌표X'].isnull()].iloc[0]  # 좌표 결측치 중 첫 번째 행 선택

# --- 4-2. 주소 조합 (도로명+시군구+번지 우선) ---
def make_address(row):
    return f"{row['시군구']} {row['도로명']}"

address = make_address(test_row)
print("테스트 주소:", address)

# --- 4-3. 지오코딩 함수로 좌표 조회 ---
lon, lat = get_coordinates_kakao(address, KAKAO_REST_API_KEY)
print("조회 결과 좌표X(경도):", lon)
print("조회 결과 좌표Y(위도):", lat)

/tmp/ipykernel_87372/3808736358.py:4: DtypeWarning: Columns (16,17,36) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv('../data/raw/train.csv')


테스트 주소: 서울특별시 강남구 개포동 삼성로 14
조회 결과 좌표X(경도): 127.068451024326
조회 결과 좌표Y(위도): 37.4861548149371


In [7]:
# --- 5. 결측치(좌표) 보간 실행 및 저장 ---

if df is not None and KAKAO_REST_API_KEY:
    # 결측치 행만 추출 (시간/비용 절감)
    df_to_geocode = df[df['좌표X'].isnull()].copy()
    if not df_to_geocode.empty:
        print(f"총 {len(df_to_geocode)}개 주소 지오코딩 시작...")

        # 주소 생성: 시군구 + 번지
        df_to_geocode['full_address'] = df_to_geocode['시군구'] + " " + df_to_geocode['번지'].astype(str)

        # tqdm progress bar와 함께 apply로 좌표 조회
        def fetch_coord(row):
            time.sleep(0.05)  # 과호출 방지(카카오 QPS 10회/1초 제한)
            return get_coordinates_kakao(row['full_address'], KAKAO_REST_API_KEY)
        
        df_to_geocode[['좌표X_new', '좌표Y_new']] = df_to_geocode.progress_apply(fetch_coord, axis=1, result_type='expand')

        # 원본 df에 좌표 결측치 채우기
        df.update(df_to_geocode[['좌표X_new', '좌표Y_new']].rename(columns={'좌표X_new': '좌표X', '좌표Y_new': '좌표Y'}))

        print("지오코딩 및 결측치 업데이트 완료.")
    else:
        print("좌표 결측치가 존재하지 않습니다.")

    # 작업 후 남은 결측치 확인
    final_missing_count = df['좌표X'].isnull().sum()
    print(f"작업 후 '좌표X' 결측치 개수: {final_missing_count}")

    # 결과 저장
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    df.to_csv(OUTPUT_PATH, index=False)
    print(f"처리 완료: '{OUTPUT_PATH}'에 저장됨.")

elif df is None:
    pass
else:
    print("[오류] .env 파일에 KAKAO_REST_API_KEY가 필요합니다.")

총 869670개 주소 지오코딩 시작...


  0%|          | 2178/869670 [04:07<27:24:06,  8.79it/s]


KeyboardInterrupt: 